In [4]:
import pandas as pd
import numpy as np

In [9]:
age = [23, 25, 26, 32]
students = np.array([['bayo',12,'m'],
            ['tobi',15,'m'],
            ['john',18,'f'],
            ['tony',22,'m']])

age_2 = np.array(age)


print(type(age_2))
print(age_2.dtype)
print(students.shape)

<class 'numpy.ndarray'>
int32
(4, 3)


In [20]:
students[:,1].astype(int).mean()

16.75

### Creating Pandas dataframe

In [22]:
# from numpy array or python list of lists
students_df = pd.DataFrame(data= students, 
                           columns = ['name', 'age', 'gender'])

students_df

,name,age,gender
0,bayo,12,m
1,tobi,15,m
2,john,18,f
3,tony,22,m


In [23]:
# from python dictioneries

staff = {
    'name': ['john','abdul','bayo','femi','tola'],
    'age': [33,23,43,21,11],
    'gender': ['male','female','male','male','female'],
    'salary': [1200,4500,4300,5500,3400]
}

staff_df = pd.DataFrame(data=staff)

In [24]:
staff_df

,name,age,gender,salary
0,john,33,male,1200
1,abdul,23,female,4500
2,bayo,43,male,4300
3,femi,21,male,5500
4,tola,11,female,3400


In [25]:
#loading a local file

orders_df = pd.read_csv('orders.csv')

In [27]:
orders_df.head(3)

,orderid,orderdate,shippeddate,requireddate,freight,staff,unitprice,quantity,companyname,city,country,companyname.1,quantity_class
0,10293,1996-08-29,1996-09-11,1996-09-26,21.18,Nancy Davolio,50.0,12,Tortuga Restaurante,Mxico D.F.,Mexico,Federal Shipping,low
1,10293,1996-08-29,1996-09-11,1996-09-26,21.18,Nancy Davolio,3.6,10,Tortuga Restaurante,Mxico D.F.,Mexico,Federal Shipping,low
2,10293,1996-08-29,1996-09-11,1996-09-26,21.18,Nancy Davolio,35.1,5,Tortuga Restaurante,Mxico D.F.,Mexico,Federal Shipping,low


In [28]:
church_df = pd.read_csv(r'C:\Users\hp\Documents\church_branch.csv')
church_df

,branch_id,branch_name,city
0,1,love,ibadan
1,2,peace,lagos
2,3,holy ghost,abuja
3,4,light,ibadan


In [37]:
# loading data directly from database 

import mysql.connector
from mysql.connector import Error
from IPython.display import clear_output


# define a function for server connection

def connect(user_name, host_name, db_name, password):
    connection = None
    try:
        connection = mysql.connector.connect(
            user = user_name,
            host = host_name,
            database = db_name,
            passwd = password
        )
        print(f'connected to database {db_name} successfully')
    except Error as err:
        print(f'Error: {err}')
        
    return connection

password = input()
clear_output()
connection = connect(user_name='root', host_name='localhost',
                     db_name='northwind', password=password)

# define a query formater function
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
    except Error as err:
        print(f'Error: {err}')
    return result

query_1 = '''

select firstname, lastname, salary
from employees
'''

# create a result set conversion function

data = read_query(connection=connection, query=query_1)

def convert_df(data, columns):
    data_list = []
    for row in data:
        row = list(row)
        data_list.append(row)
    
    data = pd.DataFrame(data = data_list, columns=columns)
    
    return data
    

connected to database northwind successfully


In [38]:
staff_df = convert_df(data=data, columns=['firstname', 'lastname', 'salary'])

staff_df

,firstname,lastname,salary
0,Nancy,Davolio,2954.55
1,Andrew,Fuller,2254.49
2,Janet,Leverling,3119.15
3,Margaret,Peacock,1861.08
4,Steven,Buchanan,1744.21
5,Michael,Suyama,2004.07
6,Robert,King,1991.55
7,Laura,Callahan,2100.50
8,Anne,Dodsworth,2333.33


In [41]:
query_2 = '''

select o.orderid, o.shipcity,s.companyname as shipper,
    concat_ws(' ', e.firstname, e.lastname) as staff,
    od.unitprice, od.quantity, o.shippeddate, o.requireddate,
    c.companyname,o.shipcountry as country,o.freight,p.productname
from `order details` od
join orders o on o.orderid = od.orderid
join employees e on e.employeeid = o.employeeid
join products p on p.productid = od.productid
join shippers s on s.shipperid = o.shipvia
join customers c on c.customerid = o.customerid
order by od.quantity desc
'''

data = read_query(connection=connection, query= query_2)

columns = ['orderid', 'city','shipper','staff', 'price',
           'quantity','ship date', 'required date','customer', 'country',
           'freight','product']

order_data = convert_df(data=data, columns=columns)

In [42]:
order_data

,orderid,city,shipper,staff,price,quantity,ship date,required date,customer,country,freight,product
0,11072,Graz,United Package,Margaret Peacock,33.2500,130,NaT,1998-06-02,Ernst Handel,Austria,258.6400,Wimmers gute Semmelkndel
1,10764,Graz,Federal Shipping,Michael Suyama,18.0000,130,1997-12-08,1997-12-31,Ernst Handel,Austria,145.4500,Chartreuse verte
2,10776,Graz,Federal Shipping,Nancy Davolio,53.0000,120,1997-12-18,1998-01-12,Ernst Handel,Austria,351.5300,Manjimup Dried Apples
3,10894,Boise,Speedy Express,Nancy Davolio,7.7500,120,1998-02-20,1998-03-18,Save-a-lot Markets,USA,116.1300,Rhnbru Klosterbier
4,10398,Boise,Federal Shipping,Andrew Fuller,19.2000,120,1997-01-09,1997-01-27,Save-a-lot Markets,USA,89.1600,Pt chinois
...,...,...,...,...,...,...,...,...,...,...,...,...
2150,10281,Madrid,Speedy Express,Margaret Peacock,7.3000,1,1996-08-21,1996-08-28,Romero y tomillo,Spain,2.9400,Teatime Chocolate Biscuits
2151,10807,Torino,Speedy Express,Margaret Peacock,18.4000,1,1998-01-30,1998-01-28,Franchi S.p.A.,Italy,1.3600,Boston Crab Meat
2152,10308,Mxico D.F.,Federal Shipping,Robert King,28.8000,1,1996-09-24,1996-10-16,Ana Trujillo Emparedados y helados,Mexico,1.6100,Gudbrandsdalsost
2153,10538,London,Federal Shipping,Anne Dodsworth,34.8000,1,1997-05-16,1997-06-12,B's Beverages,UK,4.8700,Mozzarella di Giovanni
